Задание

Берем отызывы за лето (из архива с материалами или предыдущего занятия)

1. Учим conv сеть для классификации

2. Рассмотреть 2-а варианта сеточек 

2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/

2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [ ]:
!pip install pymorphy2

In [ ]:
!pip install stop_words

In [3]:
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  
import tensorflow as tf
from keras.layers import Embedding

In [5]:
data = pd.read_excel("/content/drive/MyDrive/отзывы за лето.xls")

In [6]:
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [7]:
exclude = set(punctuation)
sw = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [8]:
data['text'] = data['Content'].apply(preprocess_text)
data = data.loc[data['Rating'] != 3]
data['target'] = (data['Rating'] > 3)*1

In [9]:
data.head(5)

,Rating,Content,Date,text,target
0,5,It just works!,2017-08-14,it just works,1
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложениеиз минус хотеть боль...,1
2,5,Отлично все,2017-08-14,отлично,1
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,зависать 1 работа антивирус ранее пользоваться...,1
4,5,"Очень удобно, работает быстро.",2017-08-14,удобно работать быстро,1


In [10]:
data.to_csv('data_ch.csv', index=False)

In [11]:
data = pd.read_csv('data_ch.csv')

In [12]:
values = {'text' : ''}
data.fillna(value=values, inplace = True)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['target'], test_size=0.2,
                                                    random_state=13, stratify=data['target'])

In [14]:
train_corpus = " ".join(X_train.values)

In [16]:
tokens = word_tokenize(train_corpus)

Отфильтруем данные
и соберём в корпус N наиболее частых токенов

In [17]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [18]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

#Модель 1 

In [61]:
max_words = 200
max_len = 20
num_classes = 2

# Training
epochs = 20
batch_size = 200
print_batch_n = 100

In [62]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [63]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'отличный',
 'хороший',
 'телефон',
 'супер']

In [64]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [65]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in X_train], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in X_test], dtype=np.int32)

In [24]:
x_train[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
      dtype=int32)

In [25]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [66]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=300, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [67]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [68]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 300)           60000     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 18, 128)           115328    
_________________________________________________________________
activation_6 (Activation)    (None, 18, 128)           0         
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1290      
_________________________________________________________________
activation_7 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                

In [69]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
72/72 [==============================] - 10s 119ms/step - loss: 0.4072 - accuracy: 0.8444 - val_loss: 0.2221 - val_accuracy: 0.8430


In [70]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

20/20 [==============================] - 0s 18ms/step - loss: 0.2159 - accuracy: 0.8468


Test score: 0.21587425470352173
Test accuracy: 0.8468354344367981


Подготовка матрицы предобученных векторов

In [71]:
embeddings_index = {}
f = open('/content/drive/MyDrive/nkrya/model.txt')
for line in f:
    values = line.split()
    word = values[0]
    word = word[:word.find('_')]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 154617 word vectors.


In [72]:
EMBEDDING_DIM = 300

In [73]:
embedding_matrix = np.zeros((max_words, EMBEDDING_DIM))
for word in tokens_filtered_top:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[tokens_filtered_top.index(word)] = embedding_vector

In [74]:
model = Sequential()
model.add(Embedding(input_dim=max_words, 
                            weights=[embedding_matrix], output_dim=EMBEDDING_DIM, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [75]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [76]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
72/72 [==============================] - 10s 122ms/step - loss: 0.4227 - accuracy: 0.8450 - val_loss: 0.3089 - val_accuracy: 0.8810


In [77]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

20/20 [==============================] - 0s 19ms/step - loss: 0.2938 - accuracy: 0.8818


Test score: 0.2938244342803955
Test accuracy: 0.8817721605300903


#Модель 2

In [94]:
max_words = 20000
max_len = 60
num_classes = 2

# Training
epochs = 20
batch_size = 200
print_batch_n = 100

In [95]:
tokens = word_tokenize(train_corpus)
tokens_filtered = [word for word in tokens if word.isalnum()]
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [96]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in X_train], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in X_test], dtype=np.int32)

In [625]:
x_train[1]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0, 468, 283, 860,   9], dtype=int32)

In [616]:
#y_train = tf.keras.utils.to_categorical(y_train, num_classes)
#y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [97]:

model1 = Sequential()
model1.add(Embedding(input_dim=max_words, output_dim=300, input_length=max_len))
model1.add(Conv1D(128, 5))
model1.add(Conv1D(50, 3))
model1.add(Conv1D(128, 2))
model1.add(Activation("relu"))
model1.add(GlobalMaxPool1D())
model1.add(Dense(10))
model1.add(Activation("relu"))
model1.add(Dense(num_classes))
model1.add(Activation('softmax'))

In [98]:
model1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [99]:
model1.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 60, 300)           6000000   
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 56, 128)           192128    
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 54, 50)            19250     
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 53, 128)           12928     
_________________________________________________________________
activation_21 (Activation)   (None, 53, 128)           0         
_________________________________________________________________
global_max_pooling1d_7 (Glob (None, 128)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)              

In [100]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model1.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
72/72 [==============================] - 37s 497ms/step - loss: 0.3879 - accuracy: 0.8566 - val_loss: 0.1908 - val_accuracy: 0.9196


In [101]:
score = model1.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

20/20 [==============================] - 2s 97ms/step - loss: 0.1838 - accuracy: 0.9180


Test score: 0.18377207219600677
Test accuracy: 0.9179747104644775


Подготовим матрицу предобученных векторов

In [102]:
EMBEDDING_DIM = 300

In [103]:
embedding_matrix = np.zeros((max_words, EMBEDDING_DIM))
for word in tokens_filtered_top:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[tokens_filtered_top.index(word)] = embedding_vector

In [110]:
model1 = Sequential()
model1.add(Embedding(input_dim=max_words, 
                            weights=[embedding_matrix], output_dim=EMBEDDING_DIM, input_length=max_len))

model1.add(Conv1D(128, 5))
model1.add(Conv1D(50, 3))
model1.add(Conv1D(128, 2))
model1.add(Activation("relu"))
model1.add(GlobalMaxPool1D())
model1.add(Dense(10))
model1.add(Activation("relu"))
model1.add(Dense(num_classes))
model1.add(Activation('softmax'))

In [111]:
model1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [112]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model1.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
72/72 [==============================] - 37s 503ms/step - loss: 0.5631 - accuracy: 0.8015 - val_loss: 0.3148 - val_accuracy: 0.8430


In [113]:
score = model1.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

20/20 [==============================] - 2s 104ms/step - loss: 0.3170 - accuracy: 0.8468


Test score: 0.3169783353805542
Test accuracy: 0.8468354344367981


Я использовала эмбеддинги НКРЯ

1 модель

loss: 0.4072 - accuracy: 0.8444 - val_loss: 0.2221 - val_accuracy: 0.8430

Test accuracy: 0.8468354344367981

НКРЯ

loss: 0.4227 - accuracy: 0.8450 - val_loss: 0.3089 - val_accuracy: 0.8810


Test accuracy: 0.8817721605300903

2 модель

loss: 0.3879 - accuracy: 0.8566 - val_loss: 0.1908 - val_accuracy: 0.9196

Test accuracy: 0.9179747104644775

НКРЯ

loss: 0.5631 - accuracy: 0.8015 - val_loss: 0.3148 - val_accuracy: 0.8430


Test accuracy: 0.8468354344367981

Честно говоря, эти результаты достаточно сложно интерпретировать

На мой взгляд самый несабильный результат получился во второй модели без эмбедд. 
Возможно достаточно легкий корпус и использование предобученных векторов в данном случае не очень оправдано


по самой предобораьотке - здесь она выполнена весьма шаблонно, использование текущих стоп слов дает обнуление многих отзывов(превращаются в нулевую строку) к тому же стоп слова вроде спасибо несутт в данном корпусе смысловую нагрузку, поэтому в идаеле надо самим составить словарь стоп слов 
ну и здесь еще есть англ слова, это тоже надо учитывать